In [1]:
import numpy as np
import pandas as pd
import ast
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Cleansing

## Users data cleansing

In [3]:
users_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/raw/users_en.csv", index_col=[0])
users_df

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,M,1
1,962099,age_18_24,income_20_40,M,0
2,1047345,age_45_54,income_40_60,F,0
3,721985,age_45_54,income_20_40,F,0
4,704055,age_35_44,income_60_90,F,0
...,...,...,...,...,...
840192,339025,age_65_inf,income_0_20,F,0
840193,983617,age_18_24,income_20_40,F,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,F,0


In [ ]:
users_df.age.unique()

In [4]:
users_df.drop_duplicates(inplace=True)
users_df.reset_index(drop=True, inplace=True)

In [ ]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840197 entries, 0 to 840196
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   840197 non-null  int64 
 1   age       826102 non-null  object
 2   income    825421 non-null  object
 3   sex       826366 non-null  object
 4   kids_flg  840197 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 32.1+ MB


In [ ]:
# Get uniq users
pd.unique(users_df['user_id']).shape

(840197,)

All users are uniq. Need to drop nan values

In [6]:
#users_df=users_df.fillna('Undefined')
users_df.dropna(inplace=True)
users_df.reset_index(drop=True, inplace=True)

In [7]:
users_df.isna().sum()

user_id     0
age         0
income      0
sex         0
kids_flg    0
dtype: int64

## Interactions data cleansing

In [30]:
inter_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/raw/interactions.csv")
inter_df

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0


In [31]:
#inter_df=inter_df.fillna('Undefined')
inter_df.dropna(inplace=True)
inter_df.reset_index(drop=True, inplace=True)

In [32]:
inter_df.isna().sum()

user_id          0
item_id          0
last_watch_dt    0
total_dur        0
watched_pct      0
dtype: int64

In [33]:
inter_df.dropna(inplace=True)
inter_df.reset_index(drop=True, inplace=True)
inter_df.isna().sum()

user_id          0
item_id          0
last_watch_dt    0
total_dur        0
watched_pct      0
dtype: int64

In [34]:
pd.unique(inter_df['user_id']).shape

(962151,)

In [35]:
pd.unique(inter_df[inter_df['user_id'].isin(users_df['user_id'])]['user_id']).shape

(730390,)

Id in inter dataser less than Id in users dataset. Need to get equal numbers of user id.

In [36]:
inter_df=inter_df[inter_df['user_id'].isin(users_df['user_id'])]
inter_df.reset_index(drop=True, inplace=True)
users_df=users_df[users_df['user_id'].isin(inter_df['user_id'])]
users_df.reset_index(drop=True, inplace=True)
print('inter uniq id: ', pd.unique(inter_df['user_id']).shape)
print('users uniq id: ', pd.unique(users_df['user_id']).shape)

inter uniq id:  (730390,)
users uniq id:  (730390,)


## Items data cleansing

In [22]:
items_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/raw/items_en.csv", index_col=[0])
items_df

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,description,keywords,actors_translated,actors_transliterated,directors_translated,transliterated
0,10711,film,Talk to her,Hable con ella,2002.0,"drama, foreign, detective, melodrama",Spain,NaN,16.0,NaN,"Marco, a journalist, interviews the famous Tor...","Talk, her, 2002, Spain, friends, love, strong,...","Adolfo Fernández, Ana Fernández, Dario Grandin...","Adol'fo Fernandes, Ana Fernandes, Dario Grandi...",Pedro Almodovar,Pedro Al'modovar
1,2508,film,Naked Peppers,Search Party,2014.0,"foreign, adventure, comedy",USA,NaN,16.0,NaN,The main character has learned not to invite h...,"Naked, Peppers, 2014, USA, friends, weddings, ...","Adam Palley, Brian Huskey, JB Smoove, Jason Ma...","Adam Palli, Brajan Haski, Dzh.B. Smuv, Dzhejso...",Scott Armstrong,Skot Armstrong
2,10716,film,Tactical force,Tactical Force,2011.0,"crime, foreign, thrillers, action, comedy",Canada,NaN,16.0,NaN,"Professional wrestler Steve Austin (""All or No...","Tactical, Force, 2011, Canada, bandits, gangst...","Adrian Holmes, Darren Shalavi, Jerry Wasserman...","Adrian Holms, Darren Shalavi, Dzherri Vasserma...",Adam P. Caltraro,Adam P. Kaltraro
3,7868,film,45 years old,45 Years,2015.0,"drama, foreign, melodrama",UK,NaN,16.0,NaN,"Charlotte Rampling, Tom Courtney, Geraldine Ja...","45, years, 2015, United Kingdom, marriage, lif...","Alexandra Riddleston-Barrett, Geraldine James,...","Aleksandra Riddlston-Barrett, Dzheral'din Dzhe...",By Andrew Hay,Endrju Hej
4,16268,film,Everything Solves in a Moment,NaN,1978.0,"drama, sport, soviet, melodrama",USSR,NaN,12.0,Lenfilm,The circle of her mentors and the most loyal f...,"Everything that decides, moment, 1978, USSR, s...","Aleksandr Abdulov, Aleksandr Demyanenko, Alexe...","Aleksandr Abdulov, Aleksandr Dem'janenko, Alek...",Victor Sadovsky,Viktor Sadovskij
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15958,6443,series,The Arctic Circle,Arctic Circle,2018.0,"drama, thriller, crime","Finland, Germany",NaN,16.0,NaN,"While chasing the poachers through the forest,...","murder, virus, crime investigation, Nordic noir","Jina Kuustonen, Maximilian Brückner, Pihla Vii...","Iina Kuustonen, Maksimilian Brjukner, Pihla Vi...",Hannu Salonen,Hannu Salonen
15959,2367,series,Hope,NaN,2020.0,"drama, action",Russia,0.0,18.0,NaN,"The reason for this split lies in Hope's past,...","Nadezhda, 2020, Russia","Victoria Isakova, Aleksandr Kuzmin, Alexei Mor...","Viktorija Isakova, Aleksandr Kuz'min, Aleksej ...",Elena Khazanova,Elena Hazanova
15960,10632,series,Collusion,Hassel,2017.0,"drama, thriller, crime",Russia,0.0,18.0,NaN,A crime drama based on the novels of Swedish d...,"Collusion, 2017, Russia","Ola Rapas, Aliette Ofeim, Wilma Leeden, Shanti...","Ola Rapas, Aliett Ofejm, Uil'ma Liden, Shanti ...","Eshref Reybroek, Amir Kamdin, Eric Eger","Eshref Rejbruk, Amir Kamdin, Erik Eger"
15961,4538,series,Among the Stones,Darklands,2019.0,"drama, sport, crime",Russia,0.0,18.0,NaN,Seventeen-year-old Damien dreams of breaking o...,"Among, Stones, 2019, Russia","Dane White O'Hara, Thomas Kane-Byrne, Judith R...","Dejn Uajt O’Hara, Tomas Kejn-Birn, Dzhudit Rod...",Conor McMahon Mark O'Connor,"Mark O’Konnor, Konor MakMahon"


In [23]:
items_df.drop_duplicates(inplace=True)
items_df.reset_index(drop=True, inplace=True)

In [14]:
#items_df.dropna(inplace=True)
#items_df.reset_index(drop=True, inplace=True)

In [15]:
items_df.isna().sum()

item_id                  0.0
content_type             0.0
title                    0.0
title_orig               0.0
release_year             0.0
genres                   0.0
countries                0.0
for_kids                 0.0
age_rating               0.0
studios                  0.0
description              0.0
keywords                 0.0
actors_translated        0.0
actors_transliterated    0.0
directors_translated     0.0
transliterated           0.0
dtype: float64

In [ ]:
print('Numbers of nan: ', items_df['title_orig'].fillna(items_df['title']).isna().sum())

Numbers of nan:  1


In [24]:
items_df['title_orig']=items_df['title_orig'].fillna(items_df['title'])

In [25]:
items_df.drop(labels=['title'], inplace=True, axis=1)
items_df.reset_index(drop=True, inplace=True)

In [26]:
items_df=items_df[items_df['item_id'].isin(inter_df['item_id'])]
items_df.reset_index(drop=True, inplace=True)

In [27]:
len(set(items_df['item_id']))

15605

In [21]:
items_df

,item_id,content_type,title_orig,release_year,genres,countries,for_kids,age_rating,studios,description,keywords,actors_translated,actors_transliterated,directors_translated,transliterated


In [ ]:
items_df=items_df.fillna('Undefined')

In [ ]:
len(set(items_df['item_id']))

15008

In [ ]:
items_df.isna().sum()

item_id                  0
content_type             0
title_orig               0
release_year             0
genres                   0
countries                0
for_kids                 0
age_rating               0
studios                  0
description              0
keywords                 0
actors_translated        0
actors_transliterated    0
directors_translated     0
transliterated           0
dtype: int64

## Check sample submission users and items id

In [ ]:
samsab_df = pd.read_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/raw/sample_submission.csv")
samsab_df

,user_id,item_id
0,3,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
1,11,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
2,29,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
3,30,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
4,33,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
...,...,...
193108,1097527,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193109,1097537,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193110,1097538,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."
193111,1097544,"[9728, 15297, 10440, 14488, 13865, 12192, 341,..."


In [ ]:
samsab_df['user_id'][inter_df['user_id'].isin(users_df['user_id'])]

0               3
1              11
2              29
3              30
4              33
           ...   
193108    1097527
193109    1097537
193110    1097538
193111    1097544
193112    1097558
Name: user_id, Length: 193113, dtype: int64

In [ ]:
items_list=list()
for char in samsab_df['item_id'].values:
  items_list.append(ast.literal_eval(char))

In [ ]:
set(np.array(items_list).ravel())

{341, 512, 3734, 4151, 9728, 10440, 12192, 13865, 14488, 15297}

In [ ]:
set(np.array(items_list).ravel()) & set(inter_df['item_id'])

{341, 512, 3734, 4151, 9728, 10440, 12192, 13865, 14488, 15297}

# Save Preprocessed Data

In [28]:
users_df.to_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/users.csv", index=False)
inter_df.to_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/interactions.csv", index=False)
items_df.to_csv(r"/content/drive/My Drive/DDH_2024/1_Task/data/preprocessed/items.csv", index=False)

In [37]:
inter_df

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
4345562,786732,4880,2021-05-12,753,0.0
4345563,546862,9673,2021-04-13,2308,49.0
4345564,697262,15297,2021-08-20,18307,63.0
4345565,384202,16197,2021-04-19,6203,100.0
